In [1]:
import time
import socket
from queue import Queue 
from threading import Thread, Condition
from ipywidgets import widgets
from IPython.display import display
import random

In [2]:
class EXP_Output:
        def __init__(self):
            #self.out = widgets.Output(layout={'border': '1px solid black'})
            #display(self.out)
            #with self.out:
            #print(\"Experiment outputs\")
    
            self.Queue_Print = Queue()
            self.thread = Thread(target=self.loop_Print)
            self.thread.start()
    
        def print(self, msg):
            text_out = time.strftime("%H:%M:%S", time.localtime()) + "> "
            self.Queue_Print.put(text_out+msg)
    
        def loop_Print(self):
            while True:
                msg = self.Queue_Print.get()
                #with self.out:
                print(msg)
                self.Queue_Print.task_done()

In [3]:
class ALE_TextInput:
    
        def __init__(self, exp_out):
            
            self.Queue_User = Queue()
            self.exp_out = exp_out
    
            # Create a text input widget
            self.Text = widgets.Text(
                            value='',
                            placeholder='Type something and press enter',
                            description='To send:',
                            disabled=False
                        )
    
            # create a button object
            self.Botton =  widgets.Button(
                                description='Submit',
                                disabled=False,
                                button_style='', 
                                tooltip='Click me to submit a string',
                            )
    
            # Define a function to handle the click action on the button
            def on_submit_button_clicked(b):
    
                global thread_running
                
                msg = self.Text.value
                #self.exp_out.print(\"ALE_0: received input: \"+str(msg))
                    
                if thread_running == True:
                    if msg == "end":
                        thread_running = False
                    
                    for i in range(20):
                        self.Queue_User.put(msg+str(i))
    
                self.Text.value = ''  # Clear the input field after submission
    
            # Attach the event to the button
            self.Botton.on_click(on_submit_button_clicked)
            
            # Display the widgets
            display(self.Text, self.Botton)
            
        # to get a message from user queue. this function can block the thread
        def get(self):
            return self.Queue_User.get()

In [4]:
class ALE_TR:
    
    def __init__(self, name, upper_Tx, lower_TR, exp_out):
        
        self.Name     = name
        self.Upper_Tx = upper_Tx
        self.Lower_TR = lower_TR
        self.exp_out  = exp_out
        
    def loop_Tx(self):
        global thread_running
        c = 0
        self.exp_out.print(self.Name + ": loop_Tx starting")
    
        while (thread_running == True):
            c = c + 1
            msg = self.Upper_Tx.get()
            text_out = self.Name + " Tx: message " + str(c) + ": " + str(msg)
            self.exp_out.print(text_out)
            self.Lower_TR.send(msg)
            
    def loop_Rx(self):
        global thread_running
        c = 0
        self.exp_out.print(self.Name + ": loop_Rx starting")
    
        while (thread_running == True):
            c = c + 1
            msg = self.Lower_TR.receive()
            text_out = self.Name + " Rx: message " + str(c) + ": " + str(msg)
            self.exp_out.print(text_out)

In [5]:
STATE_READY_TO_SEND = 0
STATE_WAITING_ACK   = 1

EVENT_UPPER_TX  = 0
EVENT_LOWER_DAT = 1
EVENT_LOWER_ACK = 2
EVENT_TIMEOUT   = 3
        
class DLE_TR_FSM:
    
    def __init__(self, name, lower_TR, exp_out):
        
        self.Name      = name
        self.Lower_TR  = lower_TR
        self.Queue_Tx  = Queue()
        self.Queue_Rx  = Queue()
        self.exp_out   = exp_out
        
        self.Queue_FSM = Queue()
        
        self.cv_TxOp = Condition()
        self.TxOp    = True

        self.State = STATE_READY_TO_SEND
        
        self.Procedure = [[self.FSM_upper_Tx, self.FSM_abnormal],
                          [self.FSM_lower_Rx, self.FSM_lower_Rx],
                          [self.FSM_abnormal, self.FSM_lower_Rx_ack],
                          [self.FSM_abnormal, self.FSM_timeout]] 
        

        self.cv_Timer      = Condition()
        self.timer_active  = False
        self.timer_counter = 0
        
        self.tx_buffer  = ''
        self.tx_seq_num = '0'
        
        self.tx_ack_num = '0'
        
        
    def loop_Tx(self):
        global thread_running
        c = 0
        self.exp_out.print(self.Name + ": loop_Tx starting")
    
        while (thread_running == True):
            with self.cv_TxOp: 
                while (self.TxOp != True): 
                    self.cv_TxOp.wait()
            
                self.TxOp = False
                c = c + 1
                msg = self.Queue_Tx.get()
                text_out = self.Name + " Tx: message " + str(c) + ": " + str(msg)
                self.exp_out.print(text_out)
                self.event_add(EVENT_UPPER_TX, msg) 

    def loop_Rx(self):
        global thread_running
        c = 0
        self.exp_out.print(self.Name + ": loop_Rx starting")
    
        while (thread_running == True):
            c = c + 1
            msg = self.Lower_TR.receive()
            text_out = self.Name + " Rx: message " + str(c) + ": " + str(msg)
            self.exp_out.print(text_out)
            
            msg_type = msg[0]
            if (msg_type == '0'):
                self.event_add(EVENT_LOWER_DAT, msg[1:])
            elif (msg_type == '1'):
                self.event_add(EVENT_LOWER_ACK, msg[1:])
            else:
                text_out = self.Name + " Rx: message type unknown " + msg_type
                self.exp_out.print(text_out)

    def receive(self):
        return self.Queue_Rx.get()
    
    def send(self, msg):
        self.Queue_Tx.put(msg)
        
    def event_add(self, ev_type, msg):
        if (isinstance(msg, str)):
            msg = msg.encode()
        
        event = ev_type.to_bytes(1, "big")+msg
        self.Queue_FSM.put(event)        
        

    def loop_FSM(self):
        global thread_running
        self.exp_out.print(self.Name + ": loop_FSM starting")
        
        while (thread_running == True):
            event = self.Queue_FSM.get()
            ev_type = event[0]
            text_out = self.Name + " FSM: state: " + str(self.State)+" event type: " + str(ev_type)
            self.exp_out.print(text_out)
            msg = event[1:].decode('utf-8')
            self.Procedure[ev_type][self.State](msg)
        
    def FSM_abnormal(self, msg):
        text_out = self.Name + " FSM: error! " + msg
        self.exp_out.print(text_out)
    
    def FSM_upper_Tx(self, msg):   
        text_out = self.Name + " FSM: to send: " + msg
        self.exp_out.print(text_out)

        self.tx_buffer = msg
        
        msg = '0' + self.tx_seq_num + msg
        self.Lower_TR.send(msg)

        self.State = STATE_WAITING_ACK
        
        with self.cv_Timer:
            self.timer_active  = True
            self.timer_counter = 10
            self.cv_Timer.notify()    
    
    def FSM_lower_Rx(self, msg):
        seq = msg[0]
        msg = msg[1:]
        
        text_out = self.Name + " FSM: received: " + msg
        self.exp_out.print(text_out)

        if (seq == self.tx_ack_num):
            self.Queue_Rx.put(msg)
            self.tx_ack_num = chr(ord(self.tx_ack_num) + 1)
            if (self.tx_ack_num == chr(ord('9') + 1)):
                self.tx_ack_num = '0'
        
        else:
            text_out  = self.Name + " FSM: received received frame " + seq
            text_out += " but expected " + self.tx_ack_num
            self.exp_out.print(text_out)
            
        self.Lower_TR.send('1'+seq)

                
    def FSM_lower_Rx_ack(self, msg):
        ack = msg[0]
        if (ack != self.tx_seq_num):
            text_out = "ACK # " + ack + " does not match local seq # " + self.tx_seq_num
            self.exp_out.print(text_out)
            return

        self.timer_active = False
        
        self.tx_seq_num = chr(ord(self.tx_seq_num) + 1)
        if (self.tx_seq_num == chr(ord('9') + 1)):
                self.tx_seq_num = '0'
        
        with self.cv_TxOp:
            self.State = STATE_READY_TO_SEND
            self.TxOp  = True
            self.cv_TxOp.notify()
            
    def FSM_timeout(self, msg):
        text_out = self.Name + " FSM: to resend frame " + self.tx_seq_num + " " + self.tx_buffer
        self.exp_out.print(text_out)
        
        msg = '0' + self.tx_seq_num + self.tx_buffer
        self.Lower_TR.send(msg)
        
        with self.cv_Timer:
            self.timer_active  = True
            self.timer_counter = 10
            self.cv_Timer.notify()

    def loop_timer(self):
        global thread_running
        self.exp_out.print(self.Name + ": loop_Timer starting")
        
        while (thread_running == True):
            with self.cv_Timer: 
                while (self.timer_active == False): 
                    self.cv_Timer.wait() 
            
            time.sleep(0.5)

            if (self.timer_counter == 0):
                self.event_add(EVENT_TIMEOUT, 'x')
                self.timer_active = False

            else:
                self.timer_counter = self.timer_counter - 1

In [6]:
class PLE_TR:

    def __init__(self, name, Socket, AP_Tx, AP_Rx, exp_out):
        
        self.Name      = name
        self.Socket    = Socket
        self.AP_Tx     = AP_Tx
        self.AP_Rx     = AP_Rx
        self.Queue_Tx  = Queue()
        self.Queue_Rx  = Queue()
        self.exp_out   = exp_out
        
    def loop_Tx(self):
        global thread_running
        self.exp_out.print(self.Name + ": loop_Tx starting")
    
        while (thread_running == True):
            msg = self.Queue_Tx.get()
            text_out = self.Name + " Tx: message: " + str(msg)
            self.exp_out.print(text_out)
            msg_bytes = str.encode(msg)
            
            rand_val = random.random()
            
            if rand_val < 0.15:  # 15% packet loss
                self.exp_out.print("packet loss")
            elif rand_val < 0.20:  # 5% packet duplication
                self.exp_out.print("packet duplication")
                # Send original packet
                self.Socket.sendto(msg_bytes, self.AP_Tx)
                # Send duplicate packet
                self.Socket.sendto(msg_bytes, self.AP_Tx)
            else:  # 80% normal transmission
                self.Socket.sendto(msg_bytes, self.AP_Tx)

    def loop_Rx(self):
        global thread_running
        global bufferSize
        self.exp_out.print(self.Name + ": loop_Rx starting")
    
        self.Socket.bind(self.AP_Rx)
        
        while (thread_running == True):
            msg_addr = self.Socket.recvfrom(bufferSize)

            # Random delay between 0 and 3 seconds
            delay_time = random.uniform(0, 3)
            time.sleep(delay_time)
            self.exp_out.print(f"delayed for {delay_time:.2f} seconds")
        
            msg  = msg_addr[0].decode('utf-8')
            addr = msg_addr[1]
            text_out = self.Name + " Rx: from " + str(addr) + ": " + str(msg)
            self.exp_out.print(text_out)
            self.Queue_Rx.put(msg)
            
    def send(self, msg):
        self.Queue_Tx.put(msg)
        
    def receive(self):
        return self.Queue_Rx.get()

In [ ]:
class MLE_TR:
    
    def __init__(self, name, lower_TR, exp_out, experiment_id):
        
        self.Name = name
        self.Lower_TR = lower_TR
        self.exp_out = exp_out
        self.experiment_id = experiment_id
        
        # Statistics tracking
        self.sent_messages = []
        self.received_messages = []
        self.delays = []
        self.duplicates = 0
        self.metrics_displayed = False
        
        # Create button widget 
        self.test_button = widgets.Button(
            description='Send 100 Messages',
            disabled=False
        )
        
        # Define callback function 
        def on_test_button_clicked(b):
            self.send_test_messages()
        
        # Attach the event to the button
        self.test_button.on_click(on_test_button_clicked)
        
        # Display the button
        display(self.test_button)
        
        self.exp_out.print(f"{self.Name}: MLE_TR ready")
        
    def send_test_messages(self):
        """Send 100 message"""
        self.exp_out.print(f"{self.Name}: Sending 100 test messages")
        
        # Reset statistics
        self.sent_messages.clear()
        self.received_messages.clear()
        self.delays.clear()
        self.duplicates = 0
        self.metrics_displayed = False
        
        # Send 100 messages 
        for msg_id in range(100):
            # Get current time with nanosecond precision 
            timestamp_ns = time.time_ns()
            
            # Create message with experiment ID, message ID, and timestamp 
            message = f"EXP:{self.experiment_id}|MSG:{msg_id}|TIME:{timestamp_ns}"
            
            # Store sent message
            self.sent_messages.append((msg_id, timestamp_ns))
            
            # Send through lower layer interface 
            self.Lower_TR.send(message)
            
        self.exp_out.print(f"{self.Name}: Completed sending 100 messages")
        
    def loop_Rx(self):
        """Receive messages from another entity"""
        global thread_running_2L, thread_running_3L
        self.exp_out.print(f"{self.Name}: loop_Rx starting")
        
        # Determine which experiment thread to follow
        def is_running():
            if self.experiment_id == 1:
                return thread_running_2L
            elif self.experiment_id == 2:
                return thread_running_3L
            else:
                return False
        
        while is_running():
            try:
                # Receive message from lower layer
                msg = self.Lower_TR.receive()
                receive_time_ns = time.time_ns()
                
                # Parse message 
                if msg.startswith("EXP:"):
                    parts = msg.split("|")
                    if len(parts) >= 3:
                        exp_id = parts[0].split(":")[1]
                        msg_id = int(parts[1].split(":")[1])
                        send_time_ns = int(parts[2].split(":")[1])
                        
                        # Only process messages from our experiment
                        if exp_id == str(self.experiment_id):
                            # Check for duplicates
                            is_duplicate = False
                            for received_msg_id, _ in self.received_messages:
                                if received_msg_id == msg_id:
                                    is_duplicate = True
                                    self.duplicates += 1
                                    break
                            
                            # Store received message
                            self.received_messages.append((msg_id, receive_time_ns))
                            
                            # Calculate delay 
                            delay_ns = receive_time_ns - send_time_ns
                            delay_ms = delay_ns / 1_000_000
                            self.delays.append(delay_ms)
                            
                            self.exp_out.print(f"{self.Name} Rx: MSG:{msg_id}, Delay:{delay_ms:.2f}ms" + 
                                             (" [DUPLICATE]" if is_duplicate else ""))
                            
                            # Check if we should display final metrics (only once)
                            unique_received = len(set(msg_id for msg_id, _ in self.received_messages))
                            if unique_received >= 80 and not self.metrics_displayed:  # Show metrics when 80+ unique messages received
                                self.metrics_displayed = True
                                self.calculate_metrics()
                    
            except Exception as e:
                self.exp_out.print(f"{self.Name}: Error in loop_Rx: {str(e)}")
                
    def calculate_metrics(self):
        """Calculate and display performance metrics"""
        self.exp_out.print(f"\n{self.Name} - FINAL PERFORMANCE METRICS:")
        
        # Calculate average delay 
        if self.delays:
            avg_delay = sum(self.delays) / len(self.delays)
            self.exp_out.print(f"Average delay: {avg_delay:.2f} ms")
        
        # Calculate message loss rate 
        sent_count = len(self.sent_messages)
        unique_received = len(set(msg_id for msg_id, _ in self.received_messages))
        
        # Only show loss rate if this node sent messages
        if sent_count > 0:
            loss_rate = ((sent_count - unique_received) / sent_count) * 100
            self.exp_out.print(f"Message loss rate: {loss_rate:.2f}%")
            self.exp_out.print(f"Messages sent: {sent_count}, Unique received: {unique_received}, Duplicates: {self.duplicates}")
        else:
            # This is a receiving-only node
            self.exp_out.print(f"Unique messages received: {unique_received}, Duplicates detected: {self.duplicates}")
        
        # Calculate message duplication rate 
        total_received = len(self.received_messages)
        if total_received > 0:
            duplication_rate = (self.duplicates / total_received) * 100
            self.exp_out.print(f"Message duplication rate: {duplication_rate:.2f}%")
            
        self.exp_out.print("-" * 50)
        
    def show_final_metrics(self):
        """Manually display final metrics"""
        if not self.metrics_displayed:
            self.metrics_displayed = True
            self.calculate_metrics()
        else:
            self.exp_out.print(f"{self.Name}: Metrics already displayed")
                


In [8]:
thread_running = False
bufferSize = 1024

exp_out = EXP_Output()

AP_local_1  = ("127.0.0.1", 30000)
AP_remote_1 = ("127.0.0.1", 31111)
Socket_1    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_1       = PLE_TR("PLE_Alice", Socket_1, AP_remote_1, AP_local_1, exp_out)

DLE_1 = DLE_TR_FSM("DLE_Alice", PLE_1, exp_out)

ALE_0 = ALE_TextInput(exp_out)
ALE_1 = ALE_TR("ALE_Alice", ALE_0, DLE_1, exp_out)

Text(value='', description='To send:', placeholder='Type something and press enter')

Button(description='Submit', style=ButtonStyle(), tooltip='Click me to submit a string')

21:46:25> ALE_Alice: loop_Tx starting
21:46:25> ALE_Alice: loop_Rx starting
21:46:25> DLE_Alice: loop_Tx starting
21:46:25> DLE_Alice: loop_Rx starting
21:46:25> PLE_Alice: loop_Tx starting
21:46:25> PLE_Alice: loop_Rx starting
21:46:25> ALE_Bob: loop_Tx starting
21:46:25> ALE_Bob: loop_Rx starting
21:46:25> DLE_Bob: loop_Tx starting
21:46:25> DLE_Bob: loop_Rx starting
21:46:25> PLE_Bob: loop_Tx starting
21:46:25> PLE_Bob: loop_Rx starting
21:46:25> DLE_Alice: loop_FSM starting
21:46:25> DLE_Bob: loop_FSM starting
21:46:25> DLE_Alice: loop_Timer starting
21:46:25> DLE_Bob: loop_Timer starting


In [9]:
AP_local_2  = ("127.0.0.1", 31111)
AP_remote_2 = ("127.0.0.1", 30000)
Socket_2    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_2       = PLE_TR("PLE_Bob", Socket_2, AP_remote_2, AP_local_2, exp_out)

DLE_2 = DLE_TR_FSM("DLE_Bob", PLE_2, exp_out)

ALE_3 = ALE_TextInput(exp_out)
ALE_2 = ALE_TR("ALE_Bob", ALE_3, DLE_2, exp_out)

Text(value='', description='To send:', placeholder='Type something and press enter')

Button(description='Submit', style=ButtonStyle(), tooltip='Click me to submit a string')

In [10]:
t1_1 = Thread(target = ALE_1.loop_Tx, args = ()) 
t2_1 = Thread(target = ALE_1.loop_Rx, args = ()) 
t3_1 = Thread(target = DLE_1.loop_Tx, args = ())
t4_1 = Thread(target = DLE_1.loop_Rx, args = ())
t5_1 = Thread(target = PLE_1.loop_Tx, args = ()) 
t6_1 = Thread(target = PLE_1.loop_Rx, args = ())
f1_1 = Thread(target = DLE_1.loop_FSM, args = ())
f2_1 = Thread(target = DLE_1.loop_timer, args = ())

t1_2 = Thread(target = ALE_2.loop_Tx, args = ()) 
t2_2 = Thread(target = ALE_2.loop_Rx, args = ()) 
t3_2 = Thread(target = DLE_2.loop_Tx, args = ())
t4_2 = Thread(target = DLE_2.loop_Rx, args = ())
t5_2 = Thread(target = PLE_2.loop_Tx, args = ()) 
t6_2 = Thread(target = PLE_2.loop_Rx, args = ()) 
f1_2 = Thread(target = DLE_2.loop_FSM, args = ())
f2_2 = Thread(target = DLE_2.loop_timer, args = ())

thread_running = True

t1_1.start()
t2_1.start()
t3_1.start()
t4_1.start()
t5_1.start()
t6_1.start()
t1_2.start()
t2_2.start()
t3_2.start()
t4_2.start()
t5_2.start()
t6_2.start()
f1_1.start()
f1_2.start()
f2_1.start()
f2_2.start()

In [11]:
# Two-layer experiment: MLE_TR -> PLE_TR
exp_out_2layer = EXP_Output()

# Node 1 (Alice) - Two layers - SENDER
AP_local_2L_1  = ("127.0.0.1", 32000)
AP_remote_2L_1 = ("127.0.0.1", 33111)
Socket_2L_1    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_2L_1       = PLE_TR("PLE_Alice_2L", Socket_2L_1, AP_remote_2L_1, AP_local_2L_1, exp_out_2layer)
MLE_2L_1       = MLE_TR("MLE_Alice_2L", PLE_2L_1, exp_out_2layer, experiment_id=1)

# Node 2 (Bob) - Two layers
AP_local_2L_2  = ("127.0.0.1", 33111)
AP_remote_2L_2 = ("127.0.0.1", 32000)
Socket_2L_2    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_2L_2       = PLE_TR("PLE_Bob_2L", Socket_2L_2, AP_remote_2L_2, AP_local_2L_2, exp_out_2layer)
MLE_2L_2       = MLE_TR("MLE_Bob_2L", PLE_2L_2, exp_out_2layer, experiment_id=1)

Button(description='Send 100 Messages', style=ButtonStyle())

21:46:25> MLE_Alice_2L: MLE_TR ready


Button(description='Send 100 Messages', style=ButtonStyle())

21:46:25> MLE_Bob_2L: MLE_TR ready
21:46:25> MLE_Alice_2L: loop_Rx starting
21:46:25> MLE_Bob_2L: loop_Rx starting
21:46:25> PLE_Alice_2L: loop_Tx starting
21:46:25> PLE_Alice_2L: loop_Rx starting
21:46:25> PLE_Bob_2L: loop_Tx starting
21:46:25> PLE_Bob_2L: loop_Rx starting
21:46:25> Two-layer experiment threads started
21:46:32> MLE_Alice_2L: Sending 100 test messages
21:46:32> MLE_Alice_2L: Completed sending 100 messages
21:46:32> PLE_Alice_2L Tx: message: EXP:1|MSG:0|TIME:1760751992527992000
21:46:32> PLE_Alice_2L Tx: message: EXP:1|MSG:1|TIME:1760751992528013000
21:46:32> packet loss
21:46:32> PLE_Alice_2L Tx: message: EXP:1|MSG:2|TIME:1760751992528016000
21:46:32> PLE_Alice_2L Tx: message: EXP:1|MSG:3|TIME:1760751992528019000
21:46:32> PLE_Alice_2L Tx: message: EXP:1|MSG:4|TIME:1760751992528022000
21:46:32> PLE_Alice_2L Tx: message: EXP:1|MSG:5|TIME:1760751992528024000
21:46:32> PLE_Alice_2L Tx: message: EXP:1|MSG:6|TIME:1760751992528027000
21:46:32> packet duplication
21:46:32> P

In [12]:
# Thread management for Two-layer experiment
def start_2layer_experiment():
    global thread_running_2L
    thread_running_2L = True
    
    # Start threads for two-layer experiment
    t_mle_rx_1_2L = Thread(target=MLE_2L_1.loop_Rx, args=())
    t_mle_rx_2_2L = Thread(target=MLE_2L_2.loop_Rx, args=())
    t_ple_tx_1_2L = Thread(target=PLE_2L_1.loop_Tx, args=())
    t_ple_rx_1_2L = Thread(target=PLE_2L_1.loop_Rx, args=())
    t_ple_tx_2_2L = Thread(target=PLE_2L_2.loop_Tx, args=())
    t_ple_rx_2_2L = Thread(target=PLE_2L_2.loop_Rx, args=())
    
    t_mle_rx_1_2L.start()
    t_mle_rx_2_2L.start()
    t_ple_tx_1_2L.start()
    t_ple_rx_1_2L.start()
    t_ple_tx_2_2L.start()
    t_ple_rx_2_2L.start()
    
    exp_out_2layer.print("Two-layer experiment threads started")
    
def stop_2layer_experiment():
    global thread_running_2L
    thread_running_2L = False
    exp_out_2layer.print("Two-layer experiment stopped")

# Initialize thread control
thread_running_2L = False

In [13]:
# STEP 2: Run this cell to start the two-layer measurement experiment
print("Starting two-layer measurement experiment...")
start_2layer_experiment()

Starting two-layer measurement experiment...


In [14]:
# Three-layer experiment: MLE_TR -> DLE_TR -> PLE_TR
exp_out_3layer = EXP_Output()

# Node 1 (Alice) - Three layers
AP_local_3L_1  = ("127.0.0.1", 34000)
AP_remote_3L_1 = ("127.0.0.1", 35111)
Socket_3L_1    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_3L_1       = PLE_TR("PLE_Alice_3L", Socket_3L_1, AP_remote_3L_1, AP_local_3L_1, exp_out_3layer)
DLE_3L_1       = DLE_TR_FSM("DLE_Alice_3L", PLE_3L_1, exp_out_3layer)
MLE_3L_1       = MLE_TR("MLE_Alice_3L", DLE_3L_1, exp_out_3layer, experiment_id=2)

# Node 2 (Bob) - Three layers
AP_local_3L_2  = ("127.0.0.1", 35111)
AP_remote_3L_2 = ("127.0.0.1", 34000)
Socket_3L_2    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_3L_2       = PLE_TR("PLE_Bob_3L", Socket_3L_2, AP_remote_3L_2, AP_local_3L_2, exp_out_3layer)
DLE_3L_2       = DLE_TR_FSM("DLE_Bob_3L", PLE_3L_2, exp_out_3layer)
MLE_3L_2       = MLE_TR("MLE_Bob_3L", DLE_3L_2, exp_out_3layer, experiment_id=2)

Button(description='Send 100 Messages', style=ButtonStyle())

21:46:25> MLE_Alice_3L: MLE_TR ready


Button(description='Send 100 Messages', style=ButtonStyle())

21:46:25> MLE_Bob_3L: MLE_TR ready
21:46:25> MLE_Alice_3L: loop_Rx starting
21:46:25> MLE_Bob_3L: loop_Rx starting
21:46:25> DLE_Alice_3L: loop_Tx starting
21:46:25> DLE_Alice_3L: loop_Rx starting
21:46:25> DLE_Alice_3L: loop_FSM starting
21:46:25> DLE_Alice_3L: loop_Timer starting
21:46:25> DLE_Bob_3L: loop_Tx starting
21:46:25> DLE_Bob_3L: loop_Rx starting
21:46:25> DLE_Bob_3L: loop_FSM starting
21:46:25> DLE_Bob_3L: loop_Timer starting
21:46:25> PLE_Alice_3L: loop_Tx starting
21:46:25> PLE_Alice_3L: loop_Rx starting
21:46:25> PLE_Bob_3L: loop_Tx starting
21:46:25> PLE_Bob_3L: loop_Rx starting
21:46:25> Three-layer experiment threads started


In [15]:
# Thread management for Three-layer experiment
def start_3layer_experiment():
    global thread_running_3L
    thread_running_3L = True
    
    # Start threads for three-layer experiment
    t_mle_rx_1_3L = Thread(target=MLE_3L_1.loop_Rx, args=())
    t_mle_rx_2_3L = Thread(target=MLE_3L_2.loop_Rx, args=())
    t_dle_tx_1_3L = Thread(target=DLE_3L_1.loop_Tx, args=())
    t_dle_rx_1_3L = Thread(target=DLE_3L_1.loop_Rx, args=())
    t_dle_fsm_1_3L = Thread(target=DLE_3L_1.loop_FSM, args=())
    t_dle_timer_1_3L = Thread(target=DLE_3L_1.loop_timer, args=())
    t_dle_tx_2_3L = Thread(target=DLE_3L_2.loop_Tx, args=())
    t_dle_rx_2_3L = Thread(target=DLE_3L_2.loop_Rx, args=())
    t_dle_fsm_2_3L = Thread(target=DLE_3L_2.loop_FSM, args=())
    t_dle_timer_2_3L = Thread(target=DLE_3L_2.loop_timer, args=())
    t_ple_tx_1_3L = Thread(target=PLE_3L_1.loop_Tx, args=())
    t_ple_rx_1_3L = Thread(target=PLE_3L_1.loop_Rx, args=())
    t_ple_tx_2_3L = Thread(target=PLE_3L_2.loop_Tx, args=())
    t_ple_rx_2_3L = Thread(target=PLE_3L_2.loop_Rx, args=())
    
    t_mle_rx_1_3L.start()
    t_mle_rx_2_3L.start()
    t_dle_tx_1_3L.start()
    t_dle_rx_1_3L.start()
    t_dle_fsm_1_3L.start()
    t_dle_timer_1_3L.start()
    t_dle_tx_2_3L.start()
    t_dle_rx_2_3L.start()
    t_dle_fsm_2_3L.start()
    t_dle_timer_2_3L.start()
    t_ple_tx_1_3L.start()
    t_ple_rx_1_3L.start()
    t_ple_tx_2_3L.start()
    t_ple_rx_2_3L.start()
    
    exp_out_3layer.print("Three-layer experiment threads started")
    
def stop_3layer_experiment():
    global thread_running_3L
    thread_running_3L = False
    exp_out_3layer.print("Three-layer experiment stopped")

# Initialize thread control
thread_running_3L = False

In [16]:
# STEP 2: Run this cell to start the three-layer measurement experiment
print("Starting three-layer measurement experiment...")
start_3layer_experiment()

Starting three-layer measurement experiment...
